In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Flatten,SimpleRNN

In [ ]:
!unzip imdb_50k_binary.zip -d imdb_50k_binary
print('-'*20+'\n Upload Completed!!\n'+'-'*20)

In [ ]:
df= pd.read_csv('imdb_50k_binary/IMDB Dataset.csv')

In [ ]:
#הכרות עם הנתונים
df.info
df.head(10)

In [ ]:
df['sentiment'] = df['sentiment']=='positive'

In [ ]:
sentences = df['review'].tolist()
labels = df['sentiment'].to_numpy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sentences,
                                                    labels,
                                                    test_size=0.2,
                                                    random_state=42)

In [ ]:
#training set
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding='pre')

max_len = train_padded.shape[1]

#testing set
test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding='pre', maxlen=max_len)

In [ ]:
type(test_padded)
y_test

In [ ]:
model = Sequential([
    Embedding(input_dim=len(word_index)+1, output_dim=16),
    SimpleRNN(32),

    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')])

model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.005),
              metrics=['accuracy'])

history = model.fit(train_padded, y_train,
                    epochs=5,
                    validation_data=(test_padded, y_test))

In [ ]:
plt.plot(history.history['accuracy'], label = 'acc')
plt.plot(history.history['val_accuracy'], label ='val')
plt.plot(history.history['loss'], label = 'test loss')
plt.plot(history.history['val_loss'], label ='val loss')
plt.legend()
print(f"max accuracy was {max(history.history['val_accuracy'])}")
plt.show()

In [ ]:
## this is for the site-https://projector.tensorflow.org/
def get_reverse_index(tokenizer):
  return tokenizer.index_word

reverse_index = get_reverse_index(tokenizer)

first_10_pairs = list(reverse_index.items())[:10]

for index, word in first_10_pairs:
  print(index, word)


embedding_layer = model.layers[0]
embeddings = embedding_layer.get_weights()[0]

np.savetxt("vectors.tsv", embeddings, delimiter="\t")

with open("metadata.tsv", "w", encoding='utf-8') as f:
  for i in range(1,112279):# len(word_index) +1): #len(word_index) +1
    word = reverse_index.get(i, "<OOV>")
    f.write(word + "\n")

from google.colab import files
files.download("vectors.tsv")

files.download("metadata.tsv")
